In [1]:
import pandas as pd 


In [12]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
df.shape

(20800, 5)

In [14]:
df.drop(columns=['id'], inplace=True)

In [15]:
df.head(1)

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1


In [54]:

df['label'].value_counts()

label
0    10387
1     9816
Name: count, dtype: int64

In [18]:
len(df['author'].unique())

4202

In [19]:
df.drop(columns=['author'], inplace=True)

In [20]:
df.columns

Index(['title', 'text', 'label'], dtype='object')

In [27]:
df.head()

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
df['title'].fillna('No title', inplace=True)


In [30]:
df['combined_text'] = df['title'] + ' ' + df['text']


In [32]:
# Preprocessing: Clean the combined text 
df['combined_text'] = df['combined_text'].str.lower()  
df['combined_text'] = df['combined_text'].str.replace('[^a-z\s]', '', regex=True)  

In [33]:
X = df['combined_text']
y = df['label'] 


In [34]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
# Convert combined text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [41]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [42]:
# Make predictions
y_pred = model.predict(X_test_tfidf)

In [43]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8386537985647117
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.99      0.86      2089
           1       0.99      0.68      0.80      1952

    accuracy                           0.84      4041
   macro avg       0.88      0.83      0.83      4041
weighted avg       0.87      0.84      0.83      4041



In [79]:
import pickle 
with open('model.pkl' ,'wb') as f:
    pickle.dump(model,f)

In [78]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming the model and vectorizer are already trained and saved as 'model' and 'vectorizer'

def predict_new_text(new_text, model, vectorizer):
    """
    Predict whether a given text is 'fake' or 'real'.
    
    Parameters:
    - new_text (str): The text to predict on.
    - model: The trained classifier model.
    - vectorizer: The trained TF-IDF vectorizer.
    
    Returns:
    - str: 'Real' or 'Fake' depending on the prediction.
    """
    # Preprocess the new text (same preprocessing applied to the training data)
    new_text_cleaned = new_text.lower()  # Convert to lowercase
    new_text_cleaned = ''.join([char for char in new_text_cleaned if char.isalpha() or char.isspace()])  # Remove non-alphabetic characters

    # Convert the new text into TF-IDF features
    new_text_tfidf = vectorizer.transform([new_text_cleaned])

    # Make the prediction using the model
    prediction = model.predict(new_text_tfidf)

    # Return the result as 'Real' or 'Fake' based on the prediction
    if prediction == 0:
        return 'Real'
    else:
        return 'Fake'


new_text = "Stock Market Hits Record High as Economic Recovery Gains Momentum"
prediction = predict_new_text(new_text, model, vectorizer)
print(f"The prediction for the new text is: {prediction}")


The prediction for the new text is: Real


In [71]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

LogisticRegression()

In [72]:

y_pred = lr.predict(X_test_tfidf)

In [73]:
accuracy_score(y_test, y_pred)

0.9524870081662955

In [74]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.96      0.95      0.95      2089
           1       0.95      0.96      0.95      1952

    accuracy                           0.95      4041
   macro avg       0.95      0.95      0.95      4041
weighted avg       0.95      0.95      0.95      4041



In [76]:

new_text = "UN Reports on Rising Global Food Insecurity Due to Conflict and Climate Change"
prediction = predict_new_text(new_text, lr, vectorizer)
print(f"The prediction for the new text is: {prediction}")

The prediction for the new text is: Fake


In [70]:
# Save the logistic regression model 

import pickle 
with open('logistic_regression.pkl' ,'wb') as f:
    pickle.dump(lr,f)

In [68]:
# Save the vectorizer model 

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer,f)

In [65]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train_tfidf, y_train)
y_pred = rfc.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)


0.9361544172234595